# Links

* Čist osnovn tutorial - https://linuxhint.com/building-your-own-network-monitor-with-pyshark/
* Github projekta - https://github.com/KimiNewt/pyshark#reading-from-a-file-using-a-display-filter (pišejo osnovne zadeve glede metod pa kako delujejo)
* Osnove - https://thepacketgeek.com/pyshark/

Zadeve ne morš poganjat v jupyter notebooku...

Ampak še kr mam lah kle kodo pa vse napisan, sam kaze se v visual studio code.

# PyShark

To je python knjižnjica, ki je wrapper za tshark (tshark je wireshark command line utility).

$ pip install pyshark

Za delovanje potrebuješ tudi imeti inštaliran tshark: https://lindevs.com/install-tshark-on-ubuntu/

Če dobimo error:

PermissionError: [Errno 13] Permission denied: '/usr/bin/dumpcap'

Solution:
$ sudo chmod +x /usr/bin/dumpcap

# Capturing packets

Paketke lahko zajemamo z dvema načinoma. Branje iz datoteke oziroma zajemanje iz network interfaca.

**Zajemanje iz CAP datoteke** 

*01_reading_from_file.py*

In [3]:
import pyshark
cap = pyshark.FileCapture('./SSHv2.cap')

for packet in cap:
    print(packet)
    print(50*"=")

<FileCapture ./mycapture.cap>


RuntimeError: This event loop is already running

Preko `cap` spremenljivke lahko sedaj iteriramo in pridobimo vsak posamezen paketek.

Če želimo hiter pregled lahko podamo parameter `only_summaries = True `

*02_only_summaries.py*

In [ ]:
import pyshark
cap = pyshark.FileCapture('./SSHv2.cap', only_summaries=True)

for packet in cap:
    print(packet)

Za hitrejši pregled lahko tudi uporabimo Wireshark filter - `display_filter` parameter.

V parameter vpišemo display filter - dokumentacija https://www.wireshark.org/docs/man-pages/wireshark-filter.html

Lahko recimo obdržimo vse paketke, ki vsebujejo ssh polje.

*03_display_filter.py*

In [ ]:
import pyshark
cap = pyshark.FileCapture('./SSHv2.cap', only_summaries=True, display_filter="ssh")

for packet in cap:
    print(packet)

---

Če želimo paketke zajemati v živo lahko to storimo z `LiveCapture` metodo.

*04_live_capture.py*

In [ ]:
import pyshark

capture = pyshark.LiveCapture(interface='wlp4s0') # Poglej da daš ta prav interface kle..
'''
$ ip r | grep default
> default via 10.1.1.1 dev wlp4s0 proto dhcp metric 600 

default interface je wlp4s0
'''
for packet in capture.sniff_continuously(packet_count=5):
    print(packet)
    print(50*"=")


Na začetku definiramo interface nato pa z metodo `sniff_continiously` začnemo z branjem paketkov. Lahko definiramo `packet_count`, kar pomeni, da bomo prebrali toliko paketkov in potem zaključili s sniffingom. Lahko pa ne definiramo in beremo paketne neprestano.

> Next, the two methods sniff() and sniff_continuously() of the capture object collect the network packets. sniff() returns to the caller as soon as all the requested packets have been collected. In contrast, sniff_continuously() delivers a single packet to the caller as soon as it was collected. This allows a live stream of the network traffic.

*05_continuous_sniffing.py*

In [ ]:
import pyshark

capture = pyshark.LiveCapture(interface='wlp4s0', only_summaries=True) # Poglej da daš ta prav interface kle..

for packet in capture.sniff_continuously():
    print(packet)

---

Znotraj paket objekta lahko najdemo vse njegove informacije. Za primer poglejmo skripto kjer filtriramo DNS query.

*06_dns_filtering.py*

In [ ]:
from struct import pack
import pyshark

capture = pyshark.LiveCapture(interface='wlp4s0') # Poglej da daš ta prav interface kle..

for packet in capture.sniff_continuously():
    #print(packet.layers)
    if "dns" in packet:
        if packet.dns.qry_name:
            if "ip" in packet:
                print('DNS Request from %s: %s' % (packet.ip.src, packet.dns.qry_name))
            elif "ipv6" in packet:
                print('DNS Request from %s: %s' % (packet.ipv6.src, packet.dns.qry_name))
        elif packet.dns.resp_name:
            if "ip" in packet:
                print('DNS Response from %s: %s' % (packet.ip.src, packet.dns.resp_name))
            elif "ipv6" in packet:
                print('DNS Response from %s: %s' % (packet.ipv6.src, packet.dns.resp_name))


        print(50*"=")

Če želimo shraniti paketke v svojo datoteko lahko to naredimo sledeče:

*07_saving_to_file.py*

In [ ]:
import pyshark

capture = pyshark.LiveCapture(interface='wlp4s0', output_file="moj.cap") # Poglej da daš ta prav interface kle..

capture.sniff(packet_count=5)
for packet in capture:
    print(packet)
    
    
cap = pyshark.FileCapture('./moj.cap')
for packet in cap:
    print(packet)
    print(50*"=")

---